In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

# Cazoo Scrapping

## Main Page

In [2]:
url = 'https://www.cazoo.co.uk/cars/'
res = requests.get(url)
res

<Response [200]>

In [3]:
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tr = tbody[2]
make_model = tr.find(attrs={'class':'vehicle-cardstyles__CustomTitle-sc-1bxv5iu-8 vEuQf'}).text
version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
dets= tr.find_all(attrs={'data-test-id':'tags'})
version = version.split()
engine = ''.join(version[0])
trim = " ".join(version[1:])
price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text

In [4]:
split = make_model.split()
split[0]

'Skoda'

In [5]:
details = None
registration = None

for span in dets:
    if "miles" in span.text:
        details = span.text

#print("details", details)
parts = details.split()
mileage = details[0]
parts

['69,660', 'miles2018', 'regManualPetrol']

In [6]:
tbody = tbody[2:]

In [7]:
vehicles = []
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    fuel_type = matches[3] if any(keyword in matches[3] for keyword in ['Petrol', 'Diesel', \
                                                            'Hybrid', 'Electric']) else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [8]:
pd.DataFrame(vehicles).head()

,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Skoda,Citigo,1.0,MPI Colour Edition,"69,660 miles",2018 reg,Manual,Petrol,"£6,650"
1,Vauxhall,Corsa,1.4i,ecoTEC Design,"77,055 miles",2019 reg,Manual,Petrol,"£6,850"
2,Peugeot,108,1.2,PureTech Allure,"60,519 miles",2018 reg,Manual,Petrol,"£6,950"
3,Smart,forfour,1.0,Pure,"30,606 miles",2018 reg,Manual,Petrol,"£7,050"
4,Fiat,500,1.2,Lounge,"36,651 miles",2018 reg,Manual,Petrol,"£7,950"


In [9]:
for i in range(2, 94):
    url = 'https://www.cazoo.co.uk/cars/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

## Audi

In [10]:
url = 'https://www.cazoo.co.uk/cars/audi/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tbody = tbody[2:]

In [11]:
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    fuel_type = matches[3] if any(keyword in matches[3] for keyword in ['Petrol', 'Diesel', \
                                                            'Hybrid', 'Electric']) else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [12]:
for i in range(2, 6):
    url = 'https://www.cazoo.co.uk/cars/audi/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

## Merc

In [13]:
url = 'https://www.cazoo.co.uk/cars/mercedes-benz/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tbody = tbody[2:]

In [14]:
tbody

[<li class="resultsstyles__ResultsListItem-sc-11kjmxu-1 jNtOCn"><div class="vehicle-cardstyles__Card-sc-1bxv5iu-0 lkFzWj"><div class="vehicle-cardstyles__InfoWrap-sc-1bxv5iu-3 bjrByV"><a class="vehicle-cardstyles__LinkWrap-sc-1bxv5iu-5 gpAIfZ" data-test-id="card-wrap-link" href="/car-details/b6e4afbd-cbd0-5974-b058-5e7ecd27bbaa/"><span class="visuallyhidden">Read more about 2017 used Mercedes-Benz C Class 2.1 C220d Sport G-Tronic+</span></a><div class="cui__sc-1gwmfgv-0 dbBZuS vehicle-cardstyles__StyledLazyImage-sc-1bxv5iu-4 dGzuGf"><noscript><img alt="2017 used Mercedes-Benz C Class 2.1 C220d Sport G-Tronic+" class="cui__sc-1gwmfgv-1 doABvC" data-test-id="noscript-image" height="550" src="https://media.cazoo.com/image/upload/c_scale,f_auto,h_405,q_auto,w_720/cazoo-imagery/car_gallery_images/WDD2052042F555365/625634f8-4ca2-469e-8131-1a704ce76f41.jpg" width="978"/></noscript></div><svg class="cui__sc-1mzrdyc-0 vehicle-cardstyles__Watermark-sc-1bxv5iu-1 ieFGkf qZJeb" data-test-id="waterm

In [15]:
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [16]:
for i in range(2, 8):
    url = 'https://www.cazoo.co.uk/cars/mercedes-benz/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

## BMW

In [17]:
url = 'https://www.cazoo.co.uk/cars/bmw/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tbody = tbody[2:]

In [18]:
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [19]:
for i in range(2, 9):
    url = 'https://www.cazoo.co.uk/cars/bmw/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

## Volkswagen

In [20]:
url = 'https://www.cazoo.co.uk/cars/volkswagen/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tbody = tbody[2:]

In [21]:
for tr in tbody:
    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
    makemod = make_model.split()
    make = makemod[0]
    model = ' '.join(makemod[1:])
    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
    version = version.split()
    engine = ''.join(version[0])
    trim = " ".join(version[1:])
    dets = tr.find_all(attrs={'data-test-id':'tags'})
    details = dets[0]
    details_str = str(details)  # Convert the first object in the list to a string
    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
    Electric)', details_str)
    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
    mileage = matches[0] if 'miles' in matches[0] else None
    registration = matches[1] if 'reg' in matches[1] else None
    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
    vehicles.append(vehicle)

In [22]:
for i in range(2, 9):
    url = 'https://www.cazoo.co.uk/cars/bmw/?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
    tbody = table.find_all('li')
    tbody = tbody[2:]
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  # Convert the first object in the list to a string
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
       
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim, 'mileage':mileage,
               'reg_year':registration, 'transmission_type':trans, 'fuel_type':fuel_type,
               'price':price
              }
        vehicles.append(vehicle)

In [23]:
cazoo_df = pd.DataFrame(vehicles)
cazoo_df

,make,model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Skoda,Citigo,1.0,MPI Colour Edition,"69,660 miles",2018 reg,Manual,Petrol,"£6,650"
1,Vauxhall,Corsa,1.4i,ecoTEC Design,"77,055 miles",2019 reg,Manual,Petrol,"£6,850"
2,Peugeot,108,1.2,PureTech Allure,"60,519 miles",2018 reg,Manual,Petrol,"£6,950"
3,Smart,forfour,1.0,Pure,"30,606 miles",2018 reg,Manual,Petrol,"£7,050"
4,Fiat,500,1.2,Lounge,"36,651 miles",2018 reg,Manual,Petrol,"£7,950"
...,...,...,...,...,...,...,...,...,...
5505,BMW,X5,3.0,30d M Sport Auto xDrive,"39,067 miles",2020 reg,Auto,Diesel,"£42,450"
5506,BMW,X3,2.0,30e 12kWh M Sport Auto xDrive,"6,006 miles",2021 reg,Auto,Diesel,"£41,950"
5507,BMW,X5,3.0,40d M Sport Auto xDrive,"30,852 miles",2018 reg,Auto,Diesel,"£32,650"
5508,BMW,X5,3.0,40d M Sport Auto xDrive,"30,297 miles",2018 reg,Auto,Diesel,"£33,800"


In [24]:
cazoo_df.to_csv('../Data/cazoo_vehicles.csv', index=False)

In [25]:
cazoo_df['make'].value_counts()

make
BMW              1009
Mercedes-Benz     566
Ford              540
Audi              396
Vauxhall          330
Volkswagen        311
Land              301
Nissan            235
Toyota            199
Peugeot           189
Mini              165
Kia               144
Hyundai           132
Seat              124
Jaguar            120
Skoda             120
Renault            97
Volvo              91
Citroen            72
Fiat               53
Lexus              37
Mitsubishi         34
MG                 32
Mazda              32
Honda              29
Suzuki             29
Tesla              28
Jeep               25
Dacia              19
DS                 12
Smart              10
Polestar            8
Abarth              6
Alfa                6
Cupra               6
Subaru              1
Infiniti            1
Porsche             1
Name: count, dtype: int64

# Cinch

In [26]:
url2 = 'https://www.cinch.co.uk/find-vehicle'
res2 = requests.get(url)
soup2 = BeautifulSoup(res.content, 'html.parser')
soup2.find(attrs={'class':'listing-results-module--list--4adde'})